<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/Copy_of_lica_ml_Xiaomei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The task is to predict the correct next sentence on the [swag dataset](https://huggingface.co/datasets/allenai/swag). You can use any base model.

In [ ]:
# Installing datasets and transformers
!pip install datasets transformers

# Installing bitsandbytes and PEFT (for LoRA usage)
!pip install bitsandbytes
!pip install peft

# Updating bitsandbytes and transformers
!pip install -U bitsandbytes transformers

# Installing accelerate for multi-GPU support or distributed training
!pip install accelerate

!pip uninstall bitsandbytes -y
!pip install -U bitsandbytes

!pip install adapter-transformers
#pip install onnxruntime
#pip install transformers[onnx]
#ip install onnxruntime-tools


  Using cached transformers-4.45.1-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.45.1-py3-none-any.whl (9.9 MB)
Using cached tokenizers-0.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.9 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.4
    Uninstalling transformers-4.43.4:
      Successfully uninstalled transformers-4.43.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
adapters 1.0.0 requires transformers~=4.43.3, but you have transformers 4.45.1 which is incompatible.
Found existing installation: bitsand

In [ ]:
!pip uninstall bitsandbytes -y
!pip install -U bitsandbytes

Found existing installation: bitsandbytes 0.44.0
Uninstalling bitsandbytes-0.44.0:
  Successfully uninstalled bitsandbytes-0.44.0
  Using cached bitsandbytes-0.44.0-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
Using cached bitsandbytes-0.44.0-py3-none-manylinux_2_24_x86_64.whl (122.4 MB)


In [ ]:
from datasets import load_dataset

# Load the SWAG dataset from Hugging Face
dataset = load_dataset("swag", "regular")

# Check the dataset splits
print(dataset)

# Access the train set, for example
train_data = dataset["train"]
print(train_data[0])  # Print the first example in the training set

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.20k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/4.81M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/4.78M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/73546 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20006 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20005 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 73546
    })
    validation: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20006
    })
    test: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20005
    })
})
{'video-id': 'anetv_jkn6uvmqwh4', 'fold-ind': '3416', 'startphrase': 'Members of the procession walk down the street holding small horn brass instruments. A drum line', 'sent1': 'Members of the procession walk down the street holding small horn brass instruments.', 'sent2': 'A drum line', 'gold-source': 'gold', 'ending0': 'passes by walking down the street playing their instruments.', 

multiple-choice task:
label: This is the correct answer, represented as an integer (0, 1, 2, or 3), corresponding to which of the four endings (ending0, ending1, ending2, or ending3).Given the context (the startphrase), the model must predict the most likely next event in a sequence by selecting one of four possible sentence endings.

In [ ]:
train_data = dataset["train"]
val_data = dataset["validation"]
test_data = dataset["test"]

# Example: Get the first 5 examples from the training data
for example in train_data.select(range(5)):
    print(example)


{'video-id': 'anetv_jkn6uvmqwh4', 'fold-ind': '3416', 'startphrase': 'Members of the procession walk down the street holding small horn brass instruments. A drum line', 'sent1': 'Members of the procession walk down the street holding small horn brass instruments.', 'sent2': 'A drum line', 'gold-source': 'gold', 'ending0': 'passes by walking down the street playing their instruments.', 'ending1': 'has heard approaching them.', 'ending2': "arrives and they're outside dancing and asleep.", 'ending3': 'turns the lead singer watches the performance.', 'label': 0}
{'video-id': 'anetv_jkn6uvmqwh4', 'fold-ind': '3417', 'startphrase': 'A drum line passes by walking down the street playing their instruments. Members of the procession', 'sent1': 'A drum line passes by walking down the street playing their instruments.', 'sent2': 'Members of the procession', 'gold-source': 'gen', 'ending0': 'are playing ping pong and celebrating one left each in quick.', 'ending1': 'wait slowly towards the cadets.

In [ ]:
for i in range(3):  # Print first 3 examples
    print(f"Start phrase: {train_data[i]['startphrase']}")
    print(f"Option 1: {train_data[i]['ending0']}")
    print(f"Option 2: {train_data[i]['ending1']}")
    print(f"Option 3: {train_data[i]['ending2']}")
    print(f"Option 4: {train_data[i]['ending3']}")
    print(f"Label: {train_data[i]['label']}")
    print("-" * 50)


Start phrase: Members of the procession walk down the street holding small horn brass instruments. A drum line
Option 1: passes by walking down the street playing their instruments.
Option 2: has heard approaching them.
Option 3: arrives and they're outside dancing and asleep.
Option 4: turns the lead singer watches the performance.
Label: 0
--------------------------------------------------
Start phrase: A drum line passes by walking down the street playing their instruments. Members of the procession
Option 1: are playing ping pong and celebrating one left each in quick.
Option 2: wait slowly towards the cadets.
Option 3: continues to play as well along the crowd along with the band being interviewed.
Option 4: continue to play marching, interspersed.
Label: 3
--------------------------------------------------
Start phrase: A group of members in green uniforms walks waving flags. Members of the procession
Option 1: pay the other coaches to cheer as people this chatter dips in lawn sh

# Section 1
Baseline - what is the baseline from the model you choose. Feel free to choose any model of your choice from the transformer library

In [ ]:
from transformers import AutoTokenizer

# Load tokenizer (using RoBERTa as an example)
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# Preprocess function to tokenize inputs
def preprocess_function(examples):
    # The startphrase is combined with each possible ending (ending0 - ending3)
    first_sentences = [[context] * 4 for context in examples["startphrase"]]
    second_sentences = [
        [examples["ending0"][i], examples["ending1"][i], examples["ending2"][i], examples["ending3"][i]]
        for i in range(len(examples["ending0"]))
    ]

    # Flatten the lists for tokenization
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # Tokenize the context and possible endings
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True, padding=True, max_length=128)

    # Reshape the input ids to match the structure needed for multiple choice (batch_size, 4, max_length)
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

# Apply the preprocessing to the train, validation, and test datasets
encoded_train_data = train_data.map(preprocess_function, batched=True)
encoded_val_data = val_data.map(preprocess_function, batched=True)
encoded_test_data = test_data.map(preprocess_function, batched=True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/73546 [00:00<?, ? examples/s]

Map:   0%|          | 0/20006 [00:00<?, ? examples/s]

Map:   0%|          | 0/20005 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

# Load tokenizer (using RoBERTa as an example)
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# Preprocess function to tokenize inputs with consistent padding
def preprocess_function(examples):
    first_sentences = [[context] * 4 for context in examples["startphrase"]]  # Repeat startphrase for all 4 endings
    second_sentences = [
        [examples["ending0"][i], examples["ending1"][i], examples["ending2"][i], examples["ending3"][i]]
        for i in range(len(examples["ending0"]))
    ]

    # Flatten the list of first and second sentences
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # Tokenize the context and possible endings with padding to max_length
    tokenized_examples = tokenizer(
        first_sentences,
        second_sentences,
        truncation=True,
        padding="max_length",  # Ensure consistent padding to max_length
        max_length=128         # Set max length to a uniform value
    )

    # Reshape the input ids to (batch_size, 4, max_length)
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

# Apply the preprocessing to the train, validation, and test datasets
encoded_train_data = train_data.map(preprocess_function, batched=True)
encoded_val_data = val_data.map(preprocess_function, batched=True)
encoded_test_data = test_data.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/73546 [00:00<?, ? examples/s]

Map:   0%|          | 0/20006 [00:00<?, ? examples/s]

Map:   0%|          | 0/20005 [00:00<?, ? examples/s]

In [ ]:
encoded_train_data1=encoded_train_data.select(range(7354))
encoded_val_data1=encoded_val_data.select(range(2000))
encoded_test_data1=encoded_test_data.select(range(2000))

In [ ]:
# Save preprocessed train, validation, and test datasets locally
encoded_train_data.save_to_disk("./encoded_train_data")
encoded_val_data.save_to_disk("./encoded_val_data")
encoded_test_data.save_to_disk("./encoded_test_data")

Saving the dataset (0/1 shards):   0%|          | 0/7354 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk

# Load the preprocessed datasets from disk
encoded_train_data = load_from_disk("./encoded_train_data")
encoded_val_data = load_from_disk("./encoded_val_data")
encoded_test_data = load_from_disk("./encoded_test_data")

In [ ]:
print(f"Train data size: {len(encoded_train_data)}")
print(f"Validation data size: {len(encoded_val_data)}")
print(f"Test data size: {len(encoded_test_data)}")

Train data size: 7354
Validation data size: 2000
Test data size: 2000


# Section 2
Fine Tuning on the model for n epochs.

In [ ]:
from transformers import AutoModelForMultipleChoice, Trainer, TrainingArguments
from datasets import load_from_disk
from tqdm import tqdm

# Load pre-trained RoBERTa model for multiple-choice tasks
model = AutoModelForMultipleChoice.from_pretrained("roberta-base")

# Freeze all layers except the last layer (classifier)
for name, param in model.named_parameters():
    if "classifier" not in name:  # Unfreeze only the classifier layer
        param.requires_grad = False  # Freeze all other layers

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",              # Output directory
    evaluation_strategy="epoch",         # Evaluate at the end of each epoch
    learning_rate=2e-5,                  # Learning rate
    per_device_train_batch_size=16,      # Training batch size per device
    per_device_eval_batch_size=16,       # Evaluation batch size per device
    num_train_epochs=3,                  # Number of epochs
    weight_decay=0.01,                   # Weight decay for regularization
    fp16=True,                           # Enable mixed precision training
    gradient_accumulation_steps=2,       # Accumulate gradients over 2 steps to simulate larger batch size
    logging_dir='./logs',                # Directory for storing logs
    logging_steps=10,                    # Log every 10 steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_data,
    eval_dataset=encoded_val_data,
)

# Use tqdm for a progress bar
progress_bar = tqdm(range(len(encoded_train_data) // training_args.per_device_train_batch_size * training_args.num_train_epochs))

# Fine-tune the model with progress bar
for epoch in range(training_args.num_train_epochs):
    trainer.train()
    progress_bar.update()

# Save the model after training
trainer.save_model("./results/finetuned-model")

# Fine-tune the model and start training
trainer.train()

# Launch TensorBoard for monitoring
# tensorboard --logdir=./logs


Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
  0%|          | 0/1377 [04:05<?, ?it/s]


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

# Section 3
Can we efficiently fine tune the model optimizing on the trainable parameters and tokens.





In [ ]:
from transformers import AdapterConfig

# Add an adapter to the model
adapter_config = AdapterConfig.load("pfeiffer")
model.add_adapter("swag_adapter", config=adapter_config)

# Train only the adapter layers
model.train_adapter("swag_adapter")

# Now use the same Trainer and training loop for efficient fine-tuning
trainer.train()


In [ ]:
!pip install adapter-transformers

In [ ]:
from transformers.adapters import AdapterConfig

ModuleNotFoundError: No module named 'transformers.adapters'

In [ ]:
!pip show transformers
!pip show adapter-transformers


Name: transformers
Version: 4.43.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: adapters, peft
Name: adapter-transformers
Version: 4.0.0
Summary: Deprecated adapter-transformers package. Use adapters package instead.
Home-page: https://github.com/adapter-hub/adapters
Author: The AdapterHub team and community contributors
Author-email: calpt@mail.de
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: adapters
Required-by: 


In [ ]:
from transformers import AdapterConfig  # Correct import after installing adapter-transformers

ImportError: cannot import name 'AdapterConfig' from 'transformers' (/usr/local/lib/python3.10/dist-packages/transformers/__init__.py)

In [ ]:
from transformers import AutoModelForMultipleChoice, Trainer, TrainingArguments, BitsAndBytesConfig
from transformers.adapters import AdapterConfig
from datasets import load_from_disk

# Load preprocessed datasets from disk
encoded_train_data = load_from_disk("./encoded_train_data")
encoded_val_data = load_from_disk("./encoded_val_data")
encoded_test_data = load_from_disk("./encoded_test_data")

# Create BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,     # Enable 8-bit quantization
    llm_int8_threshold=6.0,  # This defines a threshold for quantization
)

# Load pre-trained RoBERTa model with 8-bit quantization using BitsAndBytesConfig
model = AutoModelForMultipleChoice.from_pretrained(
    "roberta-base",
    quantization_config=bnb_config,  # Use the new config object
    device_map="auto"
)

# Add Adapter configuration
adapter_config = AdapterConfig.load("pfeiffer", reduction_factor=16)  # Fast adapter-based fine-tuning
model.add_adapter("multiple_choice_task", config=adapter_config)
model.train_adapter("multiple_choice_task")

# Define training arguments with FP16 (mixed precision) to speed up training
training_args = TrainingArguments(
    output_dir="./results",              # Output directory
    learning_rate=1e-4,                  # Adjust learning rate
    per_device_train_batch_size=16,      # Training batch size
    per_device_eval_batch_size=16,       # Evaluation batch size
    num_train_epochs=3,                  # Number of epochs
    weight_decay=0.01,                   # Weight decay for regularization
    logging_dir='./logs',                # Directory for logging
    logging_steps=10,                    # Log every 10 steps
    fp16=True,                           # Enable mixed precision training (further optimizes memory)
)

# Initialize Trainer with the quantized model and Adapter layers
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_data,
    eval_dataset=encoded_val_data,
)

# Fine-tune the model
trainer.train()

# Save the quantized and fine-tuned model
trainer.save_model("./results/quantized-finetuned-model")


ModuleNotFoundError: No module named 'transformers.adapters'

# Section 4
Can we add quantization along with efficient fine tuning

In [ ]:
from torch.quantization import quantize_dynamic

# Apply quantization to the model
quantized_model = quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Now use the Trainer to fine-tune the quantized model
trainer = Trainer(
    model=quantized_model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
)

trainer.train()


In [ ]:
from transformers import AutoModelForMultipleChoice, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_from_disk

# Load preprocessed datasets from disk
encoded_train_data = load_from_disk("./encoded_train_data")
encoded_val_data = load_from_disk("./encoded_val_data")
encoded_test_data = load_from_disk("./encoded_test_data")

# Load pre-trained RoBERTa model with 8-bit quantization
model = AutoModelForMultipleChoice.from_pretrained("roberta-base", load_in_8bit=True, device_map="auto")

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,              # Rank of the low-rank matrices
    lora_alpha=32,     # Alpha scaling factor
    target_modules=["query", "value"],  # Apply LoRA to attention modules
    lora_dropout=0.1,  # Dropout for LoRA
    bias="none",       # No bias term

)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Define training arguments with FP16 (mixed precision) to speed up training
training_args = TrainingArguments(
    output_dir="./results",              # Output directory
    learning_rate=1e-4,                  # Adjust learning rate for LoRA
    per_device_train_batch_size=16,      # Training batch size
    per_device_eval_batch_size=16,       # Evaluation batch size
    num_train_epochs=3,                  # Number of epochs
    weight_decay=0.01,                   # Weight decay for regularization
    logging_dir='./logs',                # Directory for logging
    logging_steps=10,                    # Log every 10 steps
    fp16=True,                           # Enable mixed precision training (further optimizes memory)
)

# Initialize Trainer with the quantized model and LoRA
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_data,
    eval_dataset=encoded_val_data,
)

# Fine-tune the model
trainer.train()

# Save the quantized and fine-tuned model
trainer.save_model("./results/quantized-finetuned-model")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

# Section 5
Training on multiple tasks. We want to train on video summarization, video genre detection on the same dataset

### using Shared Encoder with Task-Specific Heads, but I do suggest this approach. The better approach is to save the encoder, then d the classification task one by one.

In [ ]:
from transformers import RobertaModel, RobertaForSequenceClassification, RobertaForMultipleChoice
import torch.nn as nn

class MultiTaskModel(nn.Module):
    def __init__(self, shared_model_name="roberta-base"):
        super(MultiTaskModel, self).__init__()

        # Shared encoder (RoBERTa)
        self.shared_encoder = RobertaModel.from_pretrained(shared_model_name)

        # Task-specific heads
        self.swag_head = RobertaForMultipleChoice.from_pretrained(shared_model_name)
        self.summarization_head = RobertaForSequenceClassification.from_pretrained(shared_model_name, num_labels=2)
        self.genre_head = RobertaForSequenceClassification.from_pretrained(shared_model_name, num_labels=5)

    def forward(self, task_name, input_ids, attention_mask, token_type_ids=None):
        # Shared encoder forward pass
        shared_output = self.shared_encoder(input_ids, attention_mask, token_type_ids=token_type_ids)

        # Task-specific heads based on the task name
        if task_name == "swag":
            return self.swag_head(inputs_embeds=shared_output.last_hidden_state)
        elif task_name == "summarization":
            return self.summarization_head(inputs_embeds=shared_output.last_hidden_state)
        elif task_name == "genre":
            return self.genre_head(inputs_embeds=shared_output.last_hidden_state)


### Method2: after fine-tuning the model for SWAG (sentence prediction), I can add and fine-tune new adapters for the additional tasks of video summarization and video genre detection. Each task will have its own adapter, and I can switch between tasks easily by activating the relevant adapter.


##May require fine-tuning for each task separately.

In [ ]:
from transformers import RobertaForMultipleChoice, AdapterConfig

# Load the pre-trained RoBERTa model
model = RobertaForMultipleChoice.from_pretrained("roberta-base")

# Add adapters for multiple tasks with a shared model
adapter_config = AdapterConfig.load("pfeiffer", reduction_factor=16)

# Add adapters for the tasks
model.add_adapter("swag_adapter", config=adapter_config)           # Task 1: SWAG
model.add_adapter("summarization_adapter", config=adapter_config)  # Task 2: Video Summarization
model.add_adapter("genre_adapter", config=adapter_config)          # Task 3: Genre Detection

# Enable training for all adapters simultaneously (Parallel Multi-Task Learning)
model.train_adapter(["swag_adapter", "summarization_adapter", "genre_adapter"])


In [ ]:
from torch.utils.data import DataLoader

# Combine datasets for all tasks into a single multi-task dataset
class MultiTaskDataset:
    def __init__(self, task_datasets):
        self.datasets = task_datasets  # List of task-specific datasets
        self.task_names = list(task_datasets.keys())

    def __len__(self):
        return min([len(d) for d in self.datasets.values()])  # Smallest dataset length

    def __getitem__(self, idx):
        # Sample one batch from each task dataset
        batch = {task: self.datasets[task][idx] for task in self.task_names}
        return batch

# Load individual task datasets (already tokenized)
task_datasets = {
    "swag": encoded_train_data,              # SWAG dataset
    "summarization": video_summarization_data,  # Video summarization dataset
    "genre": video_genre_data                # Video genre detection dataset
}

# Create a multi-task dataset
multi_task_dataset = MultiTaskDataset(task_datasets)

# Create DataLoader
multi_task_dataloader = DataLoader(multi_task_dataset, batch_size=16, shuffle=True)


In [ ]:
import torch
from torch.optim import AdamW

# Define optimizer (shared across tasks)
optimizer = AdamW(model.parameters(), lr=2e-5)

# Custom training loop
def multi_task_train(model, dataloader, optimizer, num_epochs=3):
    model.train()

    for epoch in range(num_epochs):
        for batch in dataloader:
            optimizer.zero_grad()

            # Process each task separately within the same batch
            for task_name, task_data in batch.items():
                # Set the active adapter for the current task
                model.set_active_adapters(task_name + "_adapter")

                # Forward pass (compute loss for the current task)
                outputs = model(**task_data)
                loss = outputs.loss

                # Backpropagate loss
                loss.backward()

            # Optimize shared parameters and task-specific adapters
            optimizer.step()

        print(f"Epoch {epoch + 1} completed")

# Train the model on multiple tasks
multi_task_train(model, multi_task_dataloader, optimizer)


In [ ]:
# Evaluate on SWAG task
model.set_active_adapters("swag_adapter")
swag_results = trainer.predict(encoded_swag_val_data)

# Evaluate on video summarization task
model.set_active_adapters("summarization_adapter")
summarization_results = trainer.predict(encoded_summarization_val_data)

# Evaluate on genre detection task
model.set_active_adapters("genre_adapter")
genre_results = trainer.predict(encoded_genre_val_data)


In [ ]:
from transformers import Trainer, TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # Set to N as needed
    weight_decay=0.01,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
)

# Fine-tune the model
trainer.train()
